In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from PIL import Image
from tqdm import tqdm
import json
import gc
import warnings
warnings.filterwarnings("ignore")

# إعداد المسارات
AUDIO_PATH = "/kaggle/input/coughvid-wav/public_dataset"
IMG_SIZE = 64  # تقليل الحجم لتقليل استهلاك الذاكرة

# الأصناف
class_map = {"COVID-19": 0, "symptomatic": 1, "healthy": 2}

# تحويل الصوت إلى صورة طيفية
def audio_to_melspectrogram(file_path):
    y, sr = librosa.load(file_path, sr=22050)
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    fig = plt.figure(figsize=(2.24, 2.24), dpi=100)
    librosa.display.specshow(mel_spec_db, sr=sr, x_axis='time', y_axis='mel')
    plt.axis('off')
    plt.tight_layout(pad=0)
    plt.savefig("temp.png", bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    img = Image.open("temp.png").resize((IMG_SIZE, IMG_SIZE)).convert('RGB')
    return np.array(img) / 255.0

# التحضير للبيانات
X, y = [], []
max_files = 2000  # تحديد عدد الملفات لتقليل استهلاك الذاكرة
counter = 0

for file in tqdm(os.listdir(AUDIO_PATH)):
    if file.endswith(".wav") and counter < max_files:
        json_file = file.replace(".wav", ".json")
        json_path = os.path.join(AUDIO_PATH, json_file)
        wav_path = os.path.join(AUDIO_PATH, file)

        if os.path.exists(json_path):
            try:
                with open(json_path, 'r') as f:
                    data = json.load(f)
                    status = data.get("status")

                    if status in class_map:
                        img = audio_to_melspectrogram(wav_path)
                        X.append(img)
                        y.append(class_map[status])
                        counter += 1
            except:
                continue

# تجهيز البيانات
X = np.array(X)
y = to_categorical(y, num_classes=3)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# بناء النموذج
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])

# تدريب النموذج
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# حفظ النموذج
model.save("cough_model_multi.h5")

# تنظيف الذاكرة
del X, y, X_train, X_test, y_train, y_test
gc.collect()


2025-04-15 12:10:18.931491: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744719019.184081      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744719019.261873      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
100%|██████████| 55101/55101 [04:57<00:00, 185.15it/s]
I0000 00:00:1744719334.916704      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Epoch 1/10


I0000 00:00:1744719338.249169     102 service.cc:148] XLA service 0x7adea80055d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744719338.250027     102 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1744719338.499308     102 cuda_dnn.cc:529] Loaded cuDNN version 90300


47/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6968 - loss: 0.8518

I0000 00:00:1744719340.363752     102 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.6998 - loss: 0.8481 - val_accuracy: 0.7600 - val_loss: 0.7023
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7429 - loss: 0.7348 - val_accuracy: 0.7600 - val_loss: 0.6931
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7732 - loss: 0.6721 - val_accuracy: 0.7600 - val_loss: 0.7159
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7529 - loss: 0.6835 - val_accuracy: 0.7600 - val_loss: 0.6972
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7488 - loss: 0.6259 - val_accuracy: 0.7600 - val_loss: 0.7308
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7680 - loss: 0.5647 - val_accuracy: 0.7400 - val_loss: 0.7516
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7884 - loss: 0.5201 - val_accuracy: 0.7550 - val_loss: 0.8072
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8290 - loss: 0.4186 - val_accuracy: 0.7400 - val_loss: 0.8885
Ep

1045